In [18]:
import pandas as pd
import ast

# 1. Read your CSV file
# Replace 'your_data.csv' with the actual path to your file
df = pd.read_csv('../Raw-Query/Raw_CPU_Max.csv')

# 2. Parse the string data into Python dictionaries and lists
def parse_structure(text):
    try:
        return ast.literal_eval(text)
    except (ValueError, SyntaxError):
        return None

df['metric_obj'] = df['metric'].apply(parse_structure)
df['values_obj'] = df['values'].apply(parse_structure)

# 3. Extract the Node Name
# We need this to differentiate between worker1 and worker2
df['node'] = df['metric_obj'].apply(lambda x: x.get('node') if x else None)

# 4. Explode the data
# This takes the list of values [[ts, val], [ts, val]...] and creates a new row for each pair
df_exploded = df.explode('values_obj')

# 5. Extract Timestamp and CPU Value from the exploded list
df_exploded['unix_ts'] = df_exploded['values_obj'].apply(lambda x: int(x[0]))
df_exploded['cpu_val'] = df_exploded['values_obj'].apply(lambda x: float(x[1]))

# 6. Pivot the table
# This turns the 'node' values (worker1, worker2) into columns
# Index = Timestamp, Columns = Node, Values = CPU
pivot_df = df_exploded.pivot(index='unix_ts', columns='node', values='cpu_val')

# 7. Convert Timestamp to Thai Time (UTC+7)
pivot_df.index = pd.to_datetime(pivot_df.index, unit='s', utc=True)
pivot_df.index = pivot_df.index.tz_convert('Asia/Bangkok')

# 8. Rename columns to match your requirements
# We map the specific Kubernetes node names to the generic 'worker1', 'worker2'
column_mapping = {
    'aj-aung-k8s-worker1': 'worker1',
    'aj-aung-k8s-worker2': 'worker2'
}
pivot_df = pivot_df.rename(columns=column_mapping)

# 9. Clean up the final DataFrame
# Select only the columns you asked for
final_df = pivot_df[['worker1', 'worker2']].reset_index()
final_df.columns.name = None  # Remove the index name
final_df = final_df.rename(columns={'unix_ts': 'timestamp'})

# 10. Save to CSV
final_df.to_csv('Processed_CPU_Max.csv', index=False)

print("File saved successfully.")
print(final_df.head())

File saved successfully.
                  timestamp  worker1  worker2
0 2026-02-06 01:01:00+07:00      4.0      4.0
1 2026-02-06 01:02:00+07:00      4.0      4.0
2 2026-02-06 01:03:00+07:00      4.0      4.0
3 2026-02-06 01:04:00+07:00      4.0      4.0
4 2026-02-06 01:05:00+07:00      4.0      4.0
